# Process pCO2 data from R/V Svea

### Första cellen nedanför laddar bara in olika moduler/paket som du behöver för att köra resten av programmet

In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
import datahandler
import json
from read_data import Cruise
from read_data import ProcessData

### Load information about cruise start and stop

Denna information sparas i regressionlinear_regression_log.

Nedan visas vad som ligger i linear_regression_log. Om du vill ha med annan data i analysen eller ändra hur många perioder en cruise ska delas upp i gör du det genom att ändra i linear_regression_log. Det beskrivs i nästa avsnitt.

In [3]:
linear_regression_log = json.load(open("./linear_regression_log.json"))
datahandler.print_linear_regression_log(linear_regression_log)

Year      Cruise      Number  Period
2020      10_22       1       ['2020-11-11', '2020-11-18']
2020      10_23       1       ['2020-11-23', '2020-11-29']
2020      10_24       1       ['2020-12-04', '2020-12-14']
2021      10_21       1       ['2021-11-08', '2021-11-15']
2021      10_23       1       ['2021-12-06', '2021-12-13']
2021      10_23       2       ['2021-12-14', '2021-12-17']
2021      10_22       1       ['2021-11-17', '2021-11-20']
2021      10_01       1       ['2021-01-08', '2021-01-16']
2021      10_02       1       ['2021-01-19', '2021-02-04']
2021      10_03       1       ['2021-02-09', '2021-02-17']
2021      10_04       1       ['2021-03-16', '2021-03-22']
2021      10_07       1       ['2021-07-01', '2021-07-30']
2021      10_08       1       ['2021-08-01', '2021-08-30']
2021      10_09       1       ['2021-09-09', '2021-09-14']
2021      10_10       1       ['2021-09-16', '2021-09-21']
2021      10_11       1       ['2021-10-04', '2021-10-08']
2021      10_12    

### Ändra start-stop för zero drift regression

Detta ändras i linear_regression_log genom att lägga till eller ta bort en rad.

Här visas hur du ändrar i linear_regression_log.

- För att lägga till en rad:
    - *datahandler.modify_linear_regression_log(linear_regression_log, year, id, start, stop, action='add')*
- För att ta bort en rad:
    - *datahandler.modify_linear_regression_log(linear_regression_log, year, id, start, stop, action='remove')*

Linear regression log säger hur data ska delas upp efter datum och i nästa steg när data läses in kommer den att ge ett fel om data för någon period inte finns i mappen som du läser data ifrån. ¨

Anna, se näst avsnitt som beksriver vartifrån data läses och hur du ändrar sökväg eller lägger till data lokalt. 

In [ ]:
year = '2022'
id = '10_01'
start = '2021-01-03'
stop = '2021-01-14'

datahandler.modify_linear_regression_log(linear_regression_log, year, id, start, stop, action='add')
json.dump(linear_regression_log, open("./linear_regression_log.json", 'w'))
datahandler.print_linear_regression_log(linear_regression_log)

### Läs in och strukturera data från en expedition

Välj expedition genom att ändra i första raden i cellen nedan:

*my_expedition = linear_regression_log["2021"]["10_23"]['regression_periods']*

Du får fel om:

- Expeditionen inte finns i linear_regression_log får du ett fel.
- Det inte finns data för de angivna datumen i den mapp som data läses ifrån

- För att ändra vilken mapp/plats data ska läsas från så behöver du ändra i data_path nedan. Default är att det står "./example_data". Det ska ändras till den plats där du vill läsa data ifrån (kan vara lokatl eller på havgem). Till exempel "C:\Users\a001787\Script_phyton3\pCO2_Svea\pCO2_processing-main\".

Anna, om du vill läsa data lokalt utan att ändra några sökvägar så lägg till data här: C:\Users\a001787\Script_phyton3\pCO2_Svea\pCO2_processing-main\example_data\CO2FT_A 

In [15]:
cruise_key = "10_08"
my_expedition = linear_regression_log["2022"][cruise_key]['regression_periods']
data_path = "D:/data/ferrybox/2021/DeviceData"#"C:/LenaV/python3/w_pCO2/data"#"./example_data"
data_path = "C:/LenaV/python3/w_pCO2/data/2022/aug"
path_co2 = "/CO2FT_A"
path = (
    data_path
    + path_co2
)
print(path)

processed_data = 0
for cruise_period in my_expedition:
        print(cruise_period)
        try:
            cruise = Cruise(path, cruise_period)
            measurement_data, ZeroCycle = cruise.get_data_package()
            #data = ProcessData(path=path, regression_period=regression_period)
        except AssertionError as e:
            print(e)
            continue
        process = ProcessData("C:/LenaV/python3/w_pCO2/processed_data/", measurement_data, ZeroCycle, cruise_period)
        process.process_data()
        if process.valid:
            processed_data += 1
        else:
            print(f"invalid data for {year}, {cruise_key}, {cruise_period}")
        #processed_data.append(data)

C:/LenaV/python3/w_pCO2/data/2022/aug/CO2FT_A
['2022-07-12', '2022-07-18']
add filenames to filelist
find files within period 2022-07-12 to 2022-07-18
no ZeroCycle files found in C:/LenaV/python3/w_pCO2/data/2022/aug/CO2FT_A for period 2022-07-12-2022-07-18


### Läs in och strukturera data från ett år

Välja antingen detta eller en expedition ovan.

Välj år genom att ändra på första raden i cellen nedan:

*my_year = linear_regression_log["2021"]*

Du får fel om:

- Expeditionen inte finns i linear_regression_log får du ett fel.
- Det inte finns data för de angivna datumen i den mapp som data läses ifrån

Anna, lägg till data här: C:\Users\a001787\Script_phyton3\pCO2_Svea\pCO2_processing-main\example_data\CO2FT_A 

In [21]:
data_path = "D:/data/ferrybox/"#"C:/LenaV/python3/w_pCO2/data"#"./example_data"
path_co2 = "/DeviceData/CO2FT_A"
path = (
    data_path
    + path_co2
)
processed_data = 0
for year, year_log in linear_regression_log.items():
    if int(year) != 2022:
        continue
    path = data_path+str(year)+path_co2
    for cruise_key, my_expedition in year_log.items():
        for cruise_period in my_expedition["regression_periods"]:
            #if '2021-01-19' not in cruise_period:
            #    continue
            try:
                cruise = Cruise(path, cruise_period, data_save_path="C:/LenaV/python3/w_pCO2/processed_data")
                measurement_data, ZeroCycle = cruise.get_data_package()
                #data = ProcessData(path=path, regression_period=regression_period)
            except AssertionError as e:
                print(e)
                continue
            process = ProcessData(data_save_path="C:/LenaV/python3/w_pCO2/processed_data", measurements = measurement_data, zerocycles = ZeroCycle, regression_period = cruise_period)
            process.process_data()
            if process.valid:
                processed_data += 1
            else:
                print(f"invalid data for {year}, {cruise_key}, {cruise_period}")


find files within period 2022-01-03 to 2022-01-14
zerocycle rawdata saved to C:/LenaV/python3/w_pCO2/processed_data/2022-12-01/2022-01-03_2022-01-14_zeroings.txt
measurement rawdata saved to C:/LenaV/python3/w_pCO2/processed_data/2022-12-01/2022-01-03_2022-01-14_zeroings.txt
measurements_df_processed data saved to C:/LenaV/python3/w_pCO2/processed_data/2022-12-01
zerocycles_mean_df_processed data saved to C:/LenaV/python3/w_pCO2/processed_data/2022-12-01
find files within period 2022-01-20 to 2022-02-02
zerocycle rawdata saved to C:/LenaV/python3/w_pCO2/processed_data/2022-12-01/2022-01-20_2022-02-02_zeroings.txt
measurement rawdata saved to C:/LenaV/python3/w_pCO2/processed_data/2022-12-01/2022-01-20_2022-02-02_zeroings.txt
measurements_df_processed data saved to C:/LenaV/python3/w_pCO2/processed_data/2022-12-01
zerocycles_mean_df_processed data saved to C:/LenaV/python3/w_pCO2/processed_data/2022-12-01
find files within period 2022-02-08 to 2022-02-15
no ZeroCycle files found in D:/d

### Processera data

Nu har vi läst in data och ska fortsätta med att processera den så vi kan se på zero drift och korrigerad pCO2 i figurer till slut.

## Plotta och undersök data

#### Zeroing data

In the code below we will look at the zeroing data. Check how zero values look to see if more data should be removed and if and/or if the expedition should be divided into more than one regression.

- Splitting expedition: set new start and stop data and re-run calculations above. One run can hold only one expedition, i.e. one linear regression over zero values from start-stop data without breaks.

### Först laddas de moduler som behövs

In [ ]:
import plotting
import datahandler
import scipy.stats as stats
import numpy as np
import pandas as pd
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.plotting import show, save
from bokeh.io import output_notebook
from bokeh.layouts import column, row
output_notebook()

### Plotta data från zero cycles

LENA: 

- Lägg till så man ser state measurement 2 beam signal tillsammans med zero cycle two beam signal.
- Räta linjen från varje expe ska extrapoleras.



In [ ]:
import datetime as dt
import os
today = dt.datetime.today().strftime('%Y-%m-%d')
directory=f"C:/LenaV/python3/w_pCO2/processed_data/{today}/"

datalist = []
for obj in os.scandir(directory):
        if not obj.is_file():
            continue
        if not "zerocycles_mean" in obj.name:
            continue
        data = pd.read_csv(
            open(obj.path),
            sep="\t",
            parse_dates=["timestamp"],
            infer_datetime_format=True)
        datalist.append()
zerocycles_mean = pd.concat(datalist)



#### First plot zeroing data to check for drift.

In [ ]:
p = figure(
        title="title",
        x_axis_type="datetime",
        tools=["wheel_zoom", "pan", "box_zoom", "undo", "reset", "save"],
    )
p.xaxis.axis_label = "date"
p.yaxis.axis_label = "two_beam_signal"
p.legend.title = "zeroing data"
color = ['green','red','blue','purple','goldenrod','black','orange','pink', 'grey', 'saddlebrown', 'deepskyblue', 'indianred', 'yellowgreen', 'fuchsia', 'darkseagreen','navy','mediumpurple']*2
regressionperiod_no = 0

for rawdata in processed_data:
    timestamps = rawdata.zerocycles_mean_df["timestamp"].tolist()

    p.circle(
            x=rawdata.zerocycles_mean_df["datetime_objects"],
            y=rawdata.zerocycles_mean_df["two_beam_signal"],
            color= color[regressionperiod_no],
            fill_alpha=0.6,
            legend_label="{} - {}".format(rawdata.start, rawdata.stop),
            size=2,
        )

    linearcorrection_x = [
        datahandler.get_datetime_object(float(x))
        for x in timestamps]
    linearcorrection_y = rawdata.zerocycles_stats.slope*np.array(timestamps) + rawdata.zerocycles_stats.intercept

    p.line(
        x = linearcorrection_x,
        y = linearcorrection_y,
        color = color[regressionperiod_no],
        width = 1, 
        )
    regressionperiod_no += 1

p.add_layout(p.legend[0], 'right')
save(p, "C:/LenaV/python3/w_pCO2/figures/hela2021.html")

In [ ]:
p = figure(
        title="title",
        x_axis_type="datetime",
        tools=["wheel_zoom", "pan", "box_zoom", "undo", "reset", "save"],
    )
p.xaxis.axis_label = "date"
p.yaxis.axis_label = "two_beam_signal"
p.legend.title = "zeroing data"
color = ['green','red','blue','purple','goldenrod','black','orange','pink', 'grey', 'saddlebrown', 'deepskyblue', 'indianred', 'yellowgreen', 'fuchsia', 'darkseagreen','navy','mediumpurple']*2
regressionperiod_no = 0

for rawdata in processed_data:
    """rawdata.zerocycles_mean_df["datetime_objects"] = [
        datahandler.get_datetime_object(float(x))
        for x in rawdata.zerocycles_mean_df["timestamp"]
    ]"""
    timestamps = rawdata.zerocycles_mean_df["timestamp"].tolist()

    p.circle(
            x=rawdata.get_valid_data(rawdata.measurements_df_processed, validation_dict={"Q_flag": ["0: Operate"], "state": ["State_Measure"]})["datetime_objects"],
            y=rawdata.get_valid_data(rawdata.measurements_df_processed, validation_dict={"Q_flag": ["0: Operate"], "state": ["State_Measure"]})["two_beam_signal"],
            color= color[regressionperiod_no],
            fill_alpha=0,
            legend_label="raw signal in period {}".format(regressionperiod_no),
            size=4,
        )

    p.circle(
            x=rawdata.get_valid_data(rawdata.measurements_df_processed, validation_dict={"Q_flag": ["0: Operate"], "state": ["State_Measure"]})["datetime_objects"],
            y=rawdata.get_valid_data(rawdata.measurements_df_processed, validation_dict={"Q_flag": ["0: Operate"], "state": ["State_Measure"]})["drift_corrected_signal"],
            color= color[regressionperiod_no],
            fill_alpha=0.6,
            legend_label="drift corrected signal in period {}".format(regressionperiod_no),
            size=4,
        )

    p.circle(
            x=rawdata.zerocycles_mean_df["datetime_objects"],
            y=rawdata.zerocycles_mean_df["two_beam_signal"],
            color= 'black',
            fill_alpha=0.9,
            legend_label="mean of each zero cycle",
            size=2,
        )
    regressionperiod_no += 1
p.add_layout(p.legend[0], 'right')
show(p)

### Plot data from operation flag

Below we look at data from flagged with 0, meaning Operate.

In [ ]:
color = ['green','red','blue','purple','goldenrod','black','orange','pink', 'grey', 'saddlebrown', 'deepskyblue', 'indianred', 'yellowgreen', 'fuchsia', 'darkseagreen','navy','mediumpurple']*2
p = figure(
        title="Operation flag 0",
        x_axis_type="datetime",
        tools=["wheel_zoom", "pan", "box_zoom", "undo", "reset", "save"],
    )
p.xaxis.axis_label = "date"
p.yaxis.axis_label = "pCO2 corrected"
p.title = "Operation"
regressionperiod_no=0
for rawdata in processed_data:

    rawdata.measurements_df_processed["datetime_objects"] = [
        datahandler.get_datetime_object(float(x))
        for x in rawdata.measurements_df_processed["timestamp"]
    ]
    data=rawdata.get_valid_data(rawdata.measurements_df_processed, validation_dict={"Q_flag": ["0: Operate"], "State_Measure": [1]})
    source = ColumnDataSource(data)

    p.scatter(
        x="datetime_objects",
        y="pco2",
        source=source,
        color = color[regressionperiod_no],
            fill_alpha=1,
            legend_label="State Measure {}".format(regressionperiod_no),
            size=2
    )
    data=rawdata.get_valid_data(rawdata.measurements_df_processed, validation_dict={"Q_flag": ["0: Operate"], "State_Wash": [1]})
    source = ColumnDataSource(data)
    p.scatter(
        x="datetime_objects",
        y="pco2",
        source=source,
        color = color[regressionperiod_no],
            fill_alpha=0,
            legend_label="Wash state {}".format(regressionperiod_no),
            size=4
    )
    

    """p.circle(source=data,
            x="datetime_objects",
            y="PCO2_Corr",
            fill_alpha=0,
            legend_field="Q_flag",
            factors=list(set(source.data["Q_flag"])),
            #color = 'red',
            #legend_label="Processed by 4HJena without drift correction",
            size=8,
        )"""

    regressionperiod_no += 1
p.legend.click_policy = "hide"
p.add_layout(p.legend[0], 'right')

save(p, "C:/LenaV/python3/w_pCO2/figures/hela2021_operate.html")

In [ ]:
regressionperiod_no=0
data_list = []
data_list2 = []
for rawdata in processed_data:

    rawdata.measurements_df_processed["datetime_objects"] = [
        datahandler.get_datetime_object(float(x))
        for x in rawdata.measurements_df_processed["timestamp"]
    ]
    data_list.append(rawdata.get_valid_data(rawdata.measurements_df_processed, validation_dict={"State_Wash": [0], "State_Measure": [1]}))

    data_list2.append(rawdata.get_valid_data(rawdata.measurements_df_processed, validation_dict={"State_Flush": [1], "State_Zero": [1]}))

color = ['green','red','blue','purple','yellow','black','orange','pink']
p = figure(
        title="State Measure",
        x_axis_type="datetime",
        tools=["wheel_zoom", "pan", "box_zoom", "undo", "reset", "save"],
    )
p.xaxis.axis_label = "date"
p.yaxis.axis_label = "pCO2 corrected"
p.title = "State Measure"

data = pd.concat(data_list)
qflag_no = 0
for qflag, group in data.groupby("Q_flag"):
    source = ColumnDataSource(group)
    p.scatter(
        x="datetime_objects",
        y="pco2",
        source=source,
        color = color[qflag_no],
            fill_alpha=1,
            legend_label="{}".format(qflag),
            size=2
    )
    qflag_no += 1
p.legend.click_policy = "hide"
p.add_layout(p.legend[0], 'above')

p2 = figure(
        title="Not Measure",
        x_axis_type="datetime",
        tools=["wheel_zoom", "pan", "box_zoom", "undo", "reset", "save"],
        x_range=p.x_range,
        y_range=p.y_range
    )
p2.xaxis.axis_label = "date"
p2.yaxis.axis_label = "pCO2 corrected"
p2.title = "Not Measure"

data = pd.concat(data_list2)
qflag_no = 0
for qflag, group in data.groupby("Q_flag"):
    source = ColumnDataSource(group)
    p2.scatter(
        x="datetime_objects",
        y="pco2",
        source=source,
        color = color[qflag_no],
            fill_alpha=1,
            legend_label="{}".format(qflag),
            size=2
    )
    qflag_no += 1


p2.legend.click_policy = "hide"
p2.add_layout(p2.legend[0], 'above')

show(column(p,p2))